In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as Adam

from torch.utils.data import TensorDataset, DataLoader

- [ ] Entenda como os dados entram
- [ ] Entenda o que e como entra na RNN
- [ ] Entenda tudo que sai da RNN e como pode ser usado
- [ ] Alinhe código com teoria

In [82]:
inputs = torch.tensor([[0., 0.5, 0.25, 1.], [1., 0.5, 0.25, 1.]])
labels = torch.tensor([0., 1.])

In [83]:
inputs = inputs.unsqueeze(2)
labels = labels.unsqueeze(1)

In [103]:
inputs.shape

torch.Size([2, 4, 1])

In [86]:
inputs

tensor([[[0.0000],
         [0.5000],
         [0.2500],
         [1.0000]],

        [[1.0000],
         [0.5000],
         [0.2500],
         [1.0000]]])

In [87]:
dataset = TensorDataset(inputs, labels)

In [88]:
dataset.tensors

(tensor([[[0.0000],
          [0.5000],
          [0.2500],
          [1.0000]],
 
         [[1.0000],
          [0.5000],
          [0.2500],
          [1.0000]]]),
 tensor([[0.],
         [1.]]))

In [89]:
dataloader = DataLoader(dataset)

In [104]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, num_layers=1, batch_first=True)
        # self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        rnn_out, _ = self.rnn(x)
        # prediction = self.fc(rnn_out[:, -1, :])  # Apenas a saída final é usada para prever
        return rnn_out, _

In [105]:
# Definindo os parâmetros do modelo
input_size = 1  # Tamanho da entrada
hidden_size = 1  # Tamanho oculto da camada RNN
output_size = 1  # Tamanho da saída

In [106]:
model = RNNModel(input_size, hidden_size, output_size)

In [107]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [108]:
inputs[0]

tensor([[0.0000],
        [0.5000],
        [0.2500],
        [1.0000]])

In [110]:
# Before train:
# exemplo quando 0:
with torch.no_grad():
    pred = model(inputs) 
# print(f'Predicted = {pred}')

In [114]:
h_t, p = pred

In [115]:
p

tensor([[[0.3856],
         [0.3842]]])

In [167]:
EPOCHS = 1000
loss_history = []

In [168]:
optimizer.zero_grad(set_to_none=True)
for epoch in range(EPOCHS):
    # predict do modelo:
    h_t, outputs = model(inputs.float())
    
    # calcula a loss:
    loss = criterion(outputs, labels.float())

    # zera os gradientes:
    optimizer.zero_grad()
    # backpropagation
    loss.backward()
    
    # updates weights and biases
    optimizer.step()

    epoch_loss += loss.item()

epoch_loss /= len(dataloader)
loss_history.append(epoch_loss)
if (epoch + 1) % 10 == 0: 
    print(f'Época [{epoch + 1}/{EPOCHS}] - Loss: {epoch_loss:.4f}')

m:\disco m\python\pythonprojects\pytorch\env_pytorch\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([1, 2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Época [1000/1000] - Loss: 145.6864


In [169]:
torch.rand(1)

tensor([0.8635])

In [166]:
F.linear(h_t[:, -1, :], weight=torch.randn(1))

tensor([-0.5095, -0.5265], grad_fn=<MvBackward0>)

In [170]:
h_t

tensor([[[0.9494],
         [0.5100],
         [0.8192],
         [0.4688]],

        [[0.8908],
         [0.5569],
         [0.8012],
         [0.4844]]], grad_fn=<TransposeBackward1>)

In [134]:
outputs

tensor([[[0.4688],
         [0.4844]]], grad_fn=<StackBackward0>)

In [135]:
labels

tensor([[0.],
        [1.]])

In [101]:
import numpy as np

In [102]:
np.round(-3.5763e-05)

-0.0

In [223]:
rnn = nn.RNN(3, 4, 2)
input = torch.randn(4, 2, 3)
h0 = torch.randn(1*2, 2, 4)
output, hn = rnn(input, h0)

In [224]:
output

tensor([[[ 0.7350, -0.3220,  0.7996, -0.3940],
         [ 0.4268, -0.8949,  0.2960, -0.4342]],

        [[ 0.1412, -0.5637,  0.6954, -0.6535],
         [ 0.5631, -0.5364,  0.7816, -0.7101]],

        [[ 0.2250, -0.3660,  0.7170, -0.4658],
         [ 0.6490, -0.4883,  0.9131, -0.6812]],

        [[ 0.5492, -0.4261,  0.8899, -0.3753],
         [ 0.2877, -0.4899,  0.7863, -0.6831]]], grad_fn=<StackBackward0>)

In [225]:
hn

tensor([[[ 0.3562, -0.8085,  0.0298, -0.3151],
         [-0.2843, -0.3286, -0.0970, -0.7256]],

        [[ 0.5492, -0.4261,  0.8899, -0.3753],
         [ 0.2877, -0.4899,  0.7863, -0.6831]]], grad_fn=<StackBackward0>)

In [188]:
rnn  = nn.RNN(10, 20, 2)

In [189]:
input = torch.randn(5, 3, 10)

In [190]:
# [D * num_layers, N, H_out]
h0 = torch.randn(2, 3, 20)

In [192]:
output, hn = rnn(input, h0)

In [221]:
output.shape

torch.Size([5, 3, 20])

In [213]:
hn.shape

torch.Size([2, 3, 20])

In [218]:
rnn1 = nn.RNN(20, 20, 2)

In [220]:
rnn1(output, hn)[0].shape

torch.Size([5, 3, 20])